In [36]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
from sklearn.preprocessing import StandardScaler

C:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_core\_dtype.py:106: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if dtype.type == np.bool:


TypeError: 

In [ ]:
books_df = pd.read_csv(r'D:\Com Acca\FYP\e18-4yp-Reinforcement-Learning-enabled-Recommendation-System-For-Emotional-Support\code\Recommendation System\Actor Critic Model\book_filtered_data.csv')
movies_df = pd.read_csv(r'D:\Com Acca\FYP\e18-4yp-Reinforcement-Learning-enabled-Recommendation-System-For-Emotional-Support\code\Recommendation System\Actor Critic Model\movie_filtered_data.csv')
music_df = pd.read_csv(r'D:\Com Acca\FYP\e18-4yp-Reinforcement-Learning-enabled-Recommendation-System-For-Emotional-Support\code\Recommendation System\Actor Critic Model\music_filtered_data.csv')

# Display first few rows of each dataset
print(books_df.head())
print(movies_df.head())
print(music_df.head())

In [ ]:
# Preprocess the data
def preprocess_data(df):
    # Handle missing values
    df = df.fillna(method='ffill')
    
    # Identify numeric features
    numeric_features = df.select_dtypes(include=[np.number])
    
    if not numeric_features.empty:
        # Normalize numeric features
        scaler = StandardScaler()
        df[numeric_features.columns] = scaler.fit_transform(numeric_features)
    
    # One-hot encode categorical features
    df = pd.get_dummies(df)
    
    return df

# Preprocess the data
books_data = preprocess_data(books_data)
music_data = preprocess_data(music_data)
movies_data = preprocess_data(movies_data)

# Display the preprocessed data
print(books_data.head())
print(music_data.head())
print(movies_data.head())

In [30]:
class ActorCritic(tf.keras.Model):
    def __init__(self, action_size):
        super(ActorCritic, self).__init__()
        self.dense1 = layers.Dense(128, activation='relu')
        self.dense2 = layers.Dense(128, activation='relu')
        self.policy_logits = layers.Dense(action_size)
        self.values = layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        logits = self.policy_logits(x)
        values = self.values(x)
        return logits, values

model = ActorCritic(env.action_space.n)

In [ ]:
# Dummy function to get ego-centric data
def get_ego_centric_data():
    # This function should return a feature vector representing the user's environment
    # For simplicity, we return a random vector here
    return np.random.rand(5)

# Dummy function to get user preferences
def get_user_preferences(user_id):
    # This function should return a feature vector representing the user's preferences
    # For simplicity, we return a random vector here
    return np.random.rand(5)

In [ ]:
class RecommenderEnv:
    def __init__(self, user_id, music_data, books_data, movies_data, emotional_states):
        self.user_id = user_id
        self.music_data = music_data
        self.books_data = books_data
        self.movies_data = movies_data
        self.emotional_states = emotional_states
        self.state = None
        self.data = {
            'music': self.music_data,
            'books': self.books_data,
            'movies': self.movies_data
        }
        self.action_space = {
            'music': len(self.music_data),
            'books': len(self.books_data),
            'movies': len(self.movies_data)
        }

    def reset(self, specified_emotional_state=None):
        ego_centric_data = get_ego_centric_data()
        user_preferences = get_user_preferences(self.user_id)
        if specified_emotional_state is not None:
            emotional_state = specified_emotional_state
        else:
            emotional_state = np.random.choice(self.emotional_states, size=(1, len(self.emotional_states)))
        self.state = np.concatenate((emotional_state.flatten(), ego_centric_data, user_preferences))
        self.current_emotional_state = np.argmax(emotional_state)  # Store the current emotional state as an integer
        return self.state

    def step(self, action, item_type, user_feedback=None):
        action = action % self.action_space[item_type]  # Ensure the action is within the valid range
        next_state, reward, done = self._simulate_interaction(action, item_type, user_feedback)
        return next_state, reward, done

    def _simulate_interaction(self, action, item_type, user_feedback):
        next_state = self.reset()  # Reset to get the next state with new ego-centric data and user preferences
        
        if user_feedback is not None:
            reward = 1 if user_feedback == 'like' else -1
        else:
            reward = np.random.rand()
        
        done = np.random.rand() < 0.1
        return next_state, reward, done

    def get_recommendation(self, item_type):
        # Filter the activities based on the current emotional state
        valid_actions = self.data[item_type][self.data[item_type]['emotions'].apply(lambda x: self.current_emotional_state in eval(x))].index
        if len(valid_actions) == 0:
            return None
        action = np.random.choice(valid_actions)
        return action

In [ ]:
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, action_dim)
    
    def forward(self, state):
        x = torch.relu(self.fc1(state))
        action_probs = torch.softmax(self.fc2(x), dim=-1)
        return action_probs

class Critic(nn.Module):
    def __init__(self, state_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 1)
    
    def forward(self, state):
        x = torch.relu(self.fc1(state))
        value = self.fc2(x)
        return value

In [ ]:
def train_with_feedback(env, actor, critic, actor_optimizer, critic_optimizer, episodes=1000, gamma=0.99):
    item_types = ['music', 'books', 'movies']
    
    for episode in range(episodes):
        state = env.reset()
        state = torch.FloatTensor(state)
        done = False
        episode_reward = 0
        
        while not done:
            item_type = np.random.choice(item_types)
            action = env.get_recommendation(item_type)
            if action is None:
                continue
            
            action_probs = actor(state)
            m = Categorical(action_probs)
            action = m.sample()
            
            # Simulate user feedback
            user_feedback = np.random.choice(['like', 'dislike'])
            
            next_state, reward, done = env.step(action.item(), item_type, user_feedback)
            next_state = torch.FloatTensor(next_state)
            reward = torch.FloatTensor([reward])

            # Critic update
            value = critic(state)
            next_value = critic(next_state)
            target = reward + (1 - done) * gamma * next_value
            critic_loss = (target - value).pow(2).mean()
            critic_optimizer.zero_grad()
            critic_loss.backward()
            critic_optimizer.step()

            # Actor update
            advantage = target - value
            actor_loss = -m.log_prob(action) * advantage.detach()
            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()

            state = next_state
            episode_reward += reward.item()
        
        if episode % 10 == 0:
            print(f'Episode {episode}, Reward: {episode_reward}')

In [ ]:
# Example usage
emotional_states = books_data['emotions'].apply(eval).explode().unique()

ego_centric_dim = 5  # Assume ego-centric data is represented by a 5-dimensional vector
user_preferences_dim = 5  # Assume user preferences are represented by a 5-dimensional vector
env = RecommenderEnv(user_id=1, music_data=music_data, books_data=books_data, movies_data=movies_data, emotional_states=emotional_states)
state_dim = len(emotional_states) + ego_centric_dim + user_preferences_dim
action_dim = max(len(music_data), len(books_data), len(movies_data))

actor = Actor(state_dim, action_dim)
critic = Critic(state_dim)

In [ ]:
actor_optimizer = optim.Adam(actor.parameters(), lr=0.001)
critic_optimizer = optim.Adam(critic.parameters(), lr=0.001)

train_with_feedback(env, actor, critic, actor_optimizer, critic_optimizer)

In [ ]:
def get_user_feedback():
    feedback = input("Did you like the recommendation? (yes/no): ").strip().lower()
    return 'like' if feedback was 'yes' else 'dislike'

In [ ]:
def get_user_emotional_state(emotional_states):
    # Explanation of the emotional states
    print("Please enter your current emotional state:")
    for i, emotion in enumerate(emotional_states):
        print(f"{i}: {emotion}")

    mood = input("How are you feeling? (enter the number corresponding to your mood): ").strip()
    try:
        mood = int(mood)
        if mood < 0 or mood >= len(emotional_states):
            raise ValueError("Invalid mood value.")
        emotional_state = np.zeros(len(emotional_states))
        emotional_state[mood] = 1
        return emotional_state
    except ValueError:
        print("Please enter a valid number.")
        return get_user_emotional_state(emotional_states)

In [ ]:
def interactive_recommendation_system(env, actor, emotional_states):
    user_emotional_state = get_user_emotional_state(emotional_states)
    state = env.reset(specified_emotional_state=user_emotional_state)
    state = torch.FloatTensor(state)
    done = False

    while not done:
        recommendations = {}
        feedbacks = {}

        for item_type in ['music', 'books', 'movies']:
            action = env.get_recommendation(item_type)
            if action is None:
                print(f"No valid recommendations available for {item_type}.")
                continue
                
            recommendation = env.data[item_type].iloc[action]
            recommendations[item_type] = recommendation
            print(f"Recommended {item_type[:-1]}: {recommendation['name']}")

            feedbacks[item_type] = get_user_feedback()
        
        for item_type, feedback in feedbacks.items():
            next_state, reward, done = env.step(recommendations[item_type].name, item_type, feedback)
            next_state = torch.FloatTensor(next_state)
            state = next_state

        # End the loop after getting feedback for all types
        done = True

# Example usage for interactive system
interactive_recommendation_system(env, actor, emotional_states)